In [ ]:
# Set up variables for agents
# Run gcloud init if needed
import os

PROJECT_ID = "your-project-id"  # @param {type:"string"}
LOCATION = "us-west1"  # @param {type:"string"}

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION


In [ ]:
# Enable APIs
!gcloud services enable firestore.googleapis.com aiplatform.googleapis.com logging.googleapis.com storage.googleapis.com monitoring.googleapis.com cloudtrace.googleapis.com telemetry.googleapis.com

In [ ]:
# Only run once to set up database
!gcloud firestore databases create --location=nam5 --database='inventory' --quiet

In [ ]:
%pip install -r inventory_agent/requirements.txt

In [ ]:
# Run locally
!adk web

In [ ]:
# Deploy inventory agent to GCP

!adk deploy agent_engine inventory_agent --project=$PROJECT_ID --agent_engine_config_file=inventory_agent/.agent_engine_config.json

In [ ]:
# Quick hack(?) to update Service Account to have access to firestore database

PROJECT_NUMBER = !gcloud projects describe $PROJECT_ID --format="value(projectNumber)"
PROJECT_NUMBER = PROJECT_NUMBER[0]

In [ ]:
# We only need to run this once (not sure if this is created before Agent exists)

!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member="serviceAccount:service-$PROJECT_NUMBER@gcp-sa-aiplatform-re.iam.gserviceaccount.com" \
    --role="roles/datastore.user"

# We may need to redeploy the agent to have access to db

In [ ]:
import vertexai
from vertexai import agent_engines

vertexai.init(project=PROJECT_ID, location=LOCATION)

agents_list = list(agent_engines.list())
if agents_list:
    remote_agent = agents_list[0]  # Get the first (most recent) agent
    client = agent_engines
    print(f"✅ Connected to deployed agent: {remote_agent.resource_name}")
else:
    print("❌ No agents found. Please deploy first.")

In [ ]:
async for item in remote_agent.async_stream_query(
    message="What items do I have?",
    user_id="user_42",
):
    print(item)

In [ ]:
agent_engines.delete(resource_name=remote_agent.resource_name, force=True)

print("✅ Agent successfully deleted")